# Sammensetning av alt

#### Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plts
from sklearn.metrics import accuracy_score, precision_score, recall_score
from LSTM import long_short_term_memory as LSTM
from rf import rf
import warnings

#### Colors

In [2]:
rsrp_colors = ['#384959', '#6A89A7', '#88BDF2', '#BDDDFC']
rsrq_colors = ['#614419', '#B37E2E', '#DB9A39', '#FFB343']
sinr_colors = ['#135E4B', '#4CB572', '#A1D8B5', '#CCDCDB']

In [3]:
'''
    Reading the dataset and returning two pandas DataFrames:
        The first DataFrame containing every datapoint belonging to the given campaign (if any)
        The second DataFrame containing only one column - the mode (4G or 5G)
'''
def read_dataset(filename, campaign=None, features=None):
    df = pd.read_csv(filename)
    df = df.loc[:, ~df.columns.str.match('Unnamed')]
    df = df.loc[:, ~df.columns.str.match('Timestamp')]
    df = df.replace('DC', 1)
    df = df.replace('LTE', 0)
    df = df[df['campaign'].str.contains('Driving')]

    if campaign != None:
        df = df[df['campaign'] == campaign]

    if features == None:
        features = ['RSRP', 'SINR', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ', 'campaign']
    
    df, mode = df[features], df['Mode']
    return df, mode

## LSTM-prediction of signal strength indicators

In [4]:
# campaign='Interactivity_gaming_4G5G_TIM_Driving_Palasport_to_Via_Appia_End_No_Flight_Mode'

# rsrp_results = {}
# feature_selection = [['RSRP'], 
#                      ['RSRP', 'RSRQ'], ['RSRP', 'SINR'], ['RSRP', 'SINR', 'RSRQ'],
#                      ['RSRP', 'SSS_RSRP'], 
#                      ['RSRP', 'SINR', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

# for features in feature_selection:
#     used_features = ', '.join(features)
#     feats = features.copy()
#     features.append('campaign')
#     X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
#     rsrp_results[used_features] = LSTM(X=X, target_feature='RSRP', features=feats, n_steps_in=5, n_steps_out=1)

# rsrq_results = {}
# feature_selection = [['RSRQ'], 
#                      ['RSRQ', 'SINR'], ['RSRQ', 'RSRP'], ['RSRQ', 'RSRP', 'SINR'],
#                      ['RSRQ', 'SSS_RSRQ'], 
#                      ['RSRQ', 'RSRP', 'SINR', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

# for features in feature_selection:
#     used_features = ', '.join(features)
#     feats = features.copy()
#     features.append('campaign')
#     X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
#     rsrq_results[used_features] = LSTM(X=X, target_feature='RSRQ', features=feats, n_steps_in=5, n_steps_out=1)

# sinr_results = {}
# feature_selection = [['SINR'], 
#                      ['SINR', 'RSRQ'], ['SINR', 'RSRP'], ['SINR', 'RSRP', 'RSRQ'],
#                      ['SINR', 'SSS_SINR'], 
#                      ['SINR', 'RSRP', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

# for features in feature_selection:
#     used_features = ', '.join(features)
#     feats = features.copy()
#     features.append('campaign')
#     X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
#     sinr_results[used_features] = LSTM(X=X, target_feature='SINR', features=feats, n_steps_in=5, n_steps_out=2)

In [5]:
# df_rsrp_results = pd.DataFrame.from_dict(rsrp_results, orient='index')

# df_rsrp_results.reset_index(inplace=True)
# df_rsrp_results.rename(columns={'index': 'Combination'}, inplace=True)
# df_rsrp_results = df_rsrp_results.drop(['y_pred', 'y_true', 'loss', 'val_loss', 'LSTM', 'scaler_pred'], axis=1)
# df_rsrp_results

In [6]:
# df_rsrq_results = pd.DataFrame.from_dict(rsrq_results, orient='index')

# df_rsrq_results.reset_index(inplace=True)
# df_rsrq_results.rename(columns={'index': 'Combination'}, inplace=True)
# df_rsrq_results = df_rsrq_results.drop(['y_pred', 'y_true', 'loss', 'val_loss', 'LSTM', 'scaler_pred'], axis=1)
# df_rsrq_results

In [7]:
# df_sinr_results = pd.DataFrame.from_dict(sinr_results, orient='index')

# df_sinr_results.reset_index(inplace=True)
# df_sinr_results.rename(columns={'index': 'Combination'}, inplace=True)
# df_sinr_results = df_sinr_results.drop(['y_pred', 'y_true', 'loss', 'val_loss', 'LSTM', 'scaler_pred'], axis=1)
# df_sinr_results

In [8]:
campaign='Interactivity_gaming_4G5G_TIM_Driving_Palasport_to_Via_Appia_End_No_Flight_Mode'

df, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign)
test_idx = 3 * df.shape[0] // 4

train, mode_train, test, mode_test = df[0 : test_idx], mode[0 : test_idx], df[test_idx ::], mode[test_idx ::]

results = {}
target_features = ['RSRP', 'SINR', 'RSRQ']
feature_selection = [['RSRP'], ['SINR'], ['RSRQ'], 
                     ['RSRP', 'SINR'], ['RSRP', 'RSRQ'], ['SINR', 'RSRQ'], ['RSRP', 'SINR', 'RSRQ'],
                     ['RSRP', 'SSS_RSRP'], ['SINR', 'SSS_SINR'], ['RSRQ', 'SSS_RSRQ'], 
                     ['RSRP', 'SINR', 'RSRQ', 'SSS_RSRP', 'SSS_SINR', 'SSS_RSRQ']]

In [9]:
for target in target_features:
    for features in feature_selection:
        if features.__contains__(target):
            used_features = ', '.join(features)
            used_features = used_features + ' -> ' + target
            print(used_features)
            features.append('campaign')
            warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
            results[used_features] = LSTM(X=df[features], test_idx=test_idx, target_feature=target, features=features, n_steps_in=5, n_steps_out=1)
            features.remove('campaign')

RSRP -> RSRP
RSRP, SINR -> RSRP
RSRP, RSRQ -> RSRP
RSRP, SINR, RSRQ -> RSRP
RSRP, SSS_RSRP -> RSRP
RSRP, SINR, RSRQ, SSS_RSRP, SSS_SINR, SSS_RSRQ -> RSRP
SINR -> SINR
RSRP, SINR -> SINR
SINR, RSRQ -> SINR
RSRP, SINR, RSRQ -> SINR
SINR, SSS_SINR -> SINR
RSRP, SINR, RSRQ, SSS_RSRP, SSS_SINR, SSS_RSRQ -> SINR
RSRQ -> RSRQ
RSRP, RSRQ -> RSRQ
SINR, RSRQ -> RSRQ
RSRP, SINR, RSRQ -> RSRQ
RSRQ, SSS_RSRQ -> RSRQ
RSRP, SINR, RSRQ, SSS_RSRP, SSS_SINR, SSS_RSRQ -> RSRQ


In [10]:
df_results = pd.DataFrame.from_dict(results, orient='index')

df_results.reset_index(inplace=True)
df_results.rename(columns={'index': 'Combination'}, inplace=True)
df_results = df_results.drop(['y_pred', 'y_true', 'loss', 'val_loss', 'LSTM', 'scaler_pred'], axis=1)
df_results

,Combination,loss_final,val_loss_final,mad,mad?
0,RSRP -> RSRP,0.00064,0.00076,6.5,1.31
1,"RSRP, SINR -> RSRP",0.00062,0.00083,6.1,1.27
2,"RSRP, RSRQ -> RSRP",0.00062,0.00091,6.3,1.42
3,"RSRP, SINR, RSRQ -> RSRP",0.01831,0.06497,11.0,11.46
4,"RSRP, SSS_RSRP -> RSRP",0.00062,0.00046,6.3,0.87
5,"RSRP, SINR, RSRQ, SSS_RSRP, SSS_SINR, SSS_RSRQ...",0.00062,0.00067,6.3,0.99
6,SINR -> SINR,0.00088,0.00201,5.1,0.41
7,"RSRP, SINR -> SINR",0.00089,0.00194,5.7,0.50
8,"SINR, RSRQ -> SINR",0.00096,0.00198,5.0,0.44
9,"RSRP, SINR, RSRQ -> SINR",0.00087,0.00227,6.3,1.02


## Classification of mode using Random Forest

In [14]:
X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=['RSRP', 'RSRQ'])
feature_selection = [['RSRP'], ['SINR'], ['RSRQ'], ['SINR', 'RSRQ'], ['SINR', 'RSRP'], ['RSRP', 'RSRQ'], ['RSRP', 'SINR', 'RSRQ']]

rf_results = {}

print(np.sum(mode_test))
print(np.sum(mode_train))

for features in feature_selection:
    used_features = ', '.join(features)
    feats = features.copy()
    rf_results[used_features] = rf(X_train=train[features], X_test=test[features], mode_train=mode_train, mode_test=mode_test)

rf_results_df = pd.DataFrame.from_dict(rf_results, orient='index')
rf_results_df.reset_index(inplace=True)
rf_results_df.rename(columns={'index': 'Combination'}, inplace=True)
rf_results_df = rf_results_df.drop(['y_true', 'y_pred', 'model'], axis=1)
rf_results_df

24
1821


,Combination,Accuracy,Precision,Recall
0,RSRP,0.824349,0.044521,0.541667
1,SINR,0.636584,0.000000,0.000000
2,RSRQ,0.703210,0.000000,0.000000
3,"SINR, RSRQ",0.637795,0.000000,0.000000
4,"SINR, RSRP",0.586311,0.000000,0.000000
5,"RSRP, RSRQ",0.765597,0.000000,0.000000
6,"RSRP, SINR, RSRQ",0.626287,0.000000,0.000000


In [12]:
X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=['RSRP', 'RSRQ'])
feature_selection = [['SINR', 'RSRQ'], ['SINR', 'RSRP'], ['RSRP', 'RSRQ'], ['RSRP', 'SINR', 'RSRQ']]

rf_results = {}

for features in feature_selection:
    used_features = ', '.join(features)
    feats = features.copy()
    X, mode = read_dataset('datasets/Op1_merged.csv', campaign=campaign, features=features)
    metrics = rf(X, mode)
    # metrics['pred_y_pred'] = metrics['model'].predict()

rf_results_df = pd.DataFrame.from_dict(rf_results, orient='index')
rf_results_df.reset_index(inplace=True)
rf_results_df.rename(columns={'index': 'Combination'}, inplace=True)
rf_results_df = rf_results_df.drop(['y_true', 'y_pred', 'model'], axis=1)
rf_results_df

TypeError: rf() missing 2 required positional arguments: 'mode_train' and 'mode_test'